In [1]:
import sys
sys.path.append("..")

from misc import load, load_class_labels
from data_loader import data_loader;

dataset_root = "../dataset/"
handcrafted_path = dataset_root + 'handcrafted/'

train_dir = dataset_root + "train_set/"
val_dir = dataset_root + "val_set/"
val_degraded_dir = dataset_root + "val_set_degraded/"

sample_rate = 1
train_info = load(dataset_root + 'labels/train.csv', 1, sample_rate)
val_info = load(dataset_root + 'labels/validation.csv')

train_image_names = train_info.iloc[:, 0].values
val_image_names = val_info.iloc[:, 0].values

train_labels = train_info.iloc[:, 1].values
val_labels = val_info.iloc[:, 1].values
y_val = val_labels

class_labels = load_class_labels(dataset_root + 'classes.txt')

dl = data_loader(train_info, train_dir, 400, (224, 224));
dl_val = data_loader(val_info, train_dir, 400, (224, 224));


## Classification using handcrafted features

In [3]:
import numpy as np;
from handcrafted_extraction import load_all_features, load_features
frac_of_train = 1

sift_info = {
    'max_number': 50,
    'num_of_sample_kmeans': 1,
    'voc_size': 300,
    'max_iter': 500
}

gabor_obj = {
    'angles': np.arange(0, np.pi, np.pi/4),
    'lambdas': np.arange(0, 1, 0.2),
    'gammas': [0.5]
}

color = True;
lbp_info = [2];

sampling = train_info.sample(frac=frac_of_train)
train_image_names = sampling.iloc[:, 0].values
train_labels = sampling['label'].values

In [4]:
train_features, kmeans = load_all_features(handcrafted_path, train_image_names, load_color=False, load_gabor=False) 
val_features, _ = load_all_features(handcrafted_path, val_image_names, load_color=False, load_gabor=False)

100%|██████████| 23693/23693 [00:08<00:00, 2936.01it/s]


## Standard classification

Common code for features extraction from validation set

In [4]:
train_and_val_features = train_features.copy();
train_and_val_features = np.concatenate((train_and_val_features, val_features))

all_labels = train_labels.copy()
all_labels = np.concatenate((all_labels, val_labels))

In [14]:
from sklearn.preprocessing import normalize

normalized_features = normalize(train_and_val_features, axis=0)

### SVM

In [15]:
from sklearn import svm
from sklearn.model_selection import cross_validate

clf = svm.SVC(C=0.5)

Evaluation

In [16]:
scores = cross_validate(clf, normalized_features, all_labels, cv=2, scoring=['accuracy'])

In [17]:
print(scores)

{'fit_time': array([362.89848328, 397.57800555]), 'score_time': array([933.41616988, 933.09361148]), 'test_accuracy': array([0.02226688, 0.02202617])}


## Neural network classification

In [ ]:
from tensorflow import keras

model = keras.Sequential([
    keras.Input(shape=390),
    keras.layers.Dense(2000, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(2000, activation='relu'),
    keras.layers.Dense(251, activation='softmax')
])

model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)

In [ ]:
import pandas as pd
epoch_num = 10;

labels = keras.utils.to_categorical(train_labels.tolist(), num_classes=251)

for epoch in range(epoch_num):
    model.fit(train_features, labels, batch_size=100, shuffle=True, validation_split=0.1);

427/683 [=================>............] - ETA: 29s - loss: 5.5275 - accuracy: 0.0056

KeyboardInterrupt: 

In [ ]:
val_labels = keras.utils.to_categorical(val_labels.tolist(), num_classes=251);
res = model.evaluate(validation_features, val_labels)